<h1>Task 3</h1>

<h2>Imports</h2>

In [497]:
import math
import numpy as np
import pandas as pd
from random import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

## Read csv file

In [498]:
data = pd.read_csv('IrisData.txt')

## Task functions

In [499]:
# Sigmoid activation function
def sigmoid_derivative(x):
    f = 1 / (1 + np.exp(-1*x))
    return (f * (1 - f))

In [500]:
# Hyper Tangent activation function
def tanh_derivative(x):
    f = (2/(1 + np.exp(-2*x))-1)
    return (1 - (f**2))

## Creating the neural network layers

- Each layer consists of an input matrix, a weight matrix and an error list.

    - The Input layer, is the x_train or test matrix.

    - The output layer, is a (3, 1) matirx.
    
    - For each of the hidden layers the dimensions are (numOf neurons in this layer, numOf neurons in the previous layer)

In [501]:
# network contains layers in the form of dictionaries, each layer consists of:
# numpy array of layer weights.
# numpy array of layer inputs.
# net value.
# gradient value.
# All values are randomly intialized.
def nn_setup(numOf_hidden_Layers, numOf_neurons):
    # adding the number of neurons of the input layer
    numOf_neurons.insert(0, data.shape[1])
    # a list of dictionaries of numpy arrays, holding the layers weights, inputs, net and gradient value.
    network = list()
    input_layer   = {'weights':  np.random.rand(numOf_neurons[0]),
                     'inputs': np.random.rand(numOf_neurons[0]), 
                     'net': random(), 
                     'gradient':random()}
    input_layer['weights'] = input_layer['weights'].reshape(input_layer['weights'].shape[0], 1)
    input_layer['inputs'] = input_layer['inputs'].reshape(input_layer['inputs'].shape[0], 1)
    
    hidden_layers = [{'weights': np.random.rand(numOf_neurons[i], numOf_neurons[i - 1]),
                      'inputs': np.random.rand(numOf_neurons[i], numOf_neurons[i - 1]), 
                      'net': random(), 
                      'gradient': random()} for i in range(1, len(numOf_neurons))]
    
    output_layer  = {'weights':  np.random.rand(4, numOf_neurons[-1]),
                     'inputs': np.random.rand(4, numOf_neurons[-1]), 
                     'net': random(), 
                     'gradient': random()}
    
    network.append(input_layer)
    network += hidden_layers
    network.append(output_layer)
    #print(network)
    
    return network

<h2>Backpropagation</h2>

### First feed forward function

- use vector/matrix multiplication to calculate net value on each layer.
    - net  = sum(dot(layer_x, W.T))

In [502]:
# network contains layers in the form of dictionaries, each layer consists of:
# numpy array of layer weights.
# numpy array of layer inputs.
# net value.
# gradient value.
# All values are randomly intialized.
def feed_forward1(network, input_row, activation_fn, use_bias):
    # for each layer use vector/matrix multiplication to calculate the net value and update it in the network.
    network[0]['inputs'] = input_row.reshape(input_row.shape[0], 1)
    network[1]['inputs'] = input_row.reshape(input_row.shape[0], 1)
    last_output = []
    for i in range(1, len(network)):
        if not use_bias:
            network[i]['weights'][:][0] = np.zeros(network[i]['weights'].shape[1])
        # calculate the product of the current layer's weights and inputs.
        neurons_val = np.dot(network[i]['weights'], network[i]['inputs'])
        
        # apply the activation function on the cur neurons values.
        if activation_fn is 'Sigmoid':
            neurons_val = [sigmoid_derivative(val) for val in neurons_val]
        else:
            neurons_val = [tanh_derivative(val) for val in neurons_val]
            
        # calculate the net value.
        network[i]['net'] = neurons_val
        
        # the next layer input is this layer's output
        if i < len(network) - 1:
            network[i + 1]['inputs'] = neurons_val
        else:
            last_output = [1 if max(neurons_val[1:]) is neurons_val[i] else 0 for i in range(1, len(neurons_val))]
            last_output.insert(0, 1)
            
    return network, last_output

## First feed backward function

- use vector/matrix multiplication to calculate gradient value on each layer.
    - Output_layer_gradient = (intended_y - predicted_y) * d_activation_fn(net)
    - Hidden_layer_gradient_i = (gradient_(i-1) * W_i * d_activation_fn(net_i)

In [503]:
# network contains layers in the form of dictionaries, each layer consists of:
# numpy array of layer weights.
# numpy array of layer inputs.
# net value.
# gradient value.
# All values are randomly intialized.
def feed_backward(network, intended_y, predicted_y, activation_fn):
    # for each layer use vector/matrix multiplication to calculate the gradient value and update it in the network.
    # calculating gradient for output layer
    output_layer = network[len(network) - 1]
    output_layer['gradient'] = intended_y - predicted_y
    if activation_fn is 'Sigmoid':
        deriv = [sigmoid_derivative(val) for val in output_layer['net']]
    else:
        deriv = [tanh_derivative(val) for val in output_layer['net']]
    deriv = np.array(deriv).reshape(-1, 1)
    net = np.array(output_layer['net']).reshape(-1, 1)
    output_layer['gradient'].flatten()
    #np.dot(output_layer['gradient'], deriv)
    np.dot(output_layer['gradient'], net)
    network[len(network) - 1] = output_layer
    
    # calculating gradient for hidden layers
    previous_gradient = output_layer['gradient']
    for i in range(len(network) - 2, 0, -1): # step=-1
        layer = network[i]
        next_layer = network[i + 1]
        layer['gradient'] = np.dot(previous_gradient.T, next_layer['weights'])
        if activation_fn is 'Sigmoid':
            deriv = [sigmoid_derivative(val) for val in layer['net']]
        else:
            deriv = [tanh_derivative(val) for val in layer['net']]
        deriv = np.array(deriv).reshape(-1, 1)
        net = np.array(layer['net']).reshape(-1, 1)
        layer['gradient'].flatten()
        #np.dot(layer['gradient'], deriv)
        np.dot(layer['gradient'], net)
        network[i] = layer   
        previous_gradient = layer['gradient']
    return network

### Second feed forward function

- use vector/matrix multiplication to Update the weight matrix in each layer.
    - W_i = W_i + (learning_rate * gradient_i * x_i)

In [504]:
# network contains layers in the form of dictionaries, each layer consists of:
# numpy array of layer weights.
# numpy array of layer inputs.
# net value.
# gradient value.
# All values are randomly intialized.
def feed_forward2(network, learning_rate):
     # for each layer use vector/matrix multiplication to calculate the new weights value and update it in the network.
    for i in range (1 , len(network)):
        layer = network[i]
        net = layer['net'] * layer['gradient'] * learning_rate
        net = net.reshape(-1, 1)
        for neuron_index in range(layer['weights'].shape[0]):
            layer['weights'][neuron_index] += net[neuron_index]
        network[i] = layer
    return network

### Backpropagation model

- For each epoch call:

    - Feed forward, calculating the net values for each layer.

    - Feed Backward, calculating the gradient values for each layer.

    - Feed forward, updating the weights for each layer.

In [505]:
def backpropagation(x_train, y_train, network, learning_rate, epochs, use_bias, activation_fn):
    # for each epoch:
    for i in range(epochs):
        for ind in range(x_train.shape[0]):
            # call feed_forward1 with the given network, row of data, activation function and use_bias.
            network, y_prediction = feed_forward1(network, x_train[ind], activation_fn, use_bias)
            # call feed_backward with the returned network, cur row of y_train, cur y_prediction for this row and the activation_fn
            network = feed_backward(network, y_train[ind], np.array(y_prediction), activation_fn)
            # call feed_forward2 with the returned network and other necessary values
            network = feed_forward2(network, learning_rate)
    return network

## Testing

In [506]:
def test(x_test, y_test, network, activation_fn, use_bias):
    y_prediction = []
    for row in x_test:
        network, y = feed_forward1(network, row, activation_fn, use_bias)
        y_prediction.append(y)
    print(y_prediction)
    # calculating the accuracy.
    comparison = (y_prediction == y_test)
    co = 0
    for l in comparison:
        ans = True
        for val in l:
            ans &= val
        if ans == True:
            co += 1
    accuracy = (co/y_test.shape[0]) * 100
    return y_prediction, accuracy

## Extracting the data

In [507]:
# We will be using all the 4 feauters and 3 classes.
# The y column should be on hot encoded, meaning that if the label is c1 
    # then it should be represented as follow, 100 and so on.

def extract_data():
    data_x = data.iloc[:, :4]
    x0 = np.ones([150, 1]) # feature 0 for bias
    data_x = np.append(x0, data_x, axis=1)
    # One hot encoding the ouput column.
    values = np.array(data['Class'])
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    data_y = onehot_encoded
    data_y = np.append(x0, data_y, axis=1)
    
    x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.4, shuffle=True, stratify = data_y)
    return x_train, y_train, x_test, y_test


## Main

In [508]:
def main(numOf_hidden_layers, numOf_nuerons, activation_fn, learning_rate, epochs, use_bias):
    # Get the train and test data.
    x_train, y_train, x_test, y_test = extract_data()
    
    # Setup the Neural Network layer.
    network = nn_setup(numOf_hidden_layers, numOf_nuerons)
    
    print(network)
    
    # Call the backpropagation model and return the learned weights.
    network = backpropagation(x_train, y_train, network, learning_rate, epochs, use_bias, activation_fn)

    print(network)
    
    # Test the moddel and return its accuracy, then print it.
    y_prediction_test, accuracy_test = test(x_test, y_test, network, activation_fn, use_bias)
    print('Testing accuracy:\n', accuracy_test)
    
    # print the confusion matrix.
    print('Confusion Matrix:\n', confusion_matrix(y_test, y_prediction_test))
    return

<h1>UI</h1>

- text box to take the number of desired hidden layers.

- text box to take the number of desired neurons in each hidden layer.

- combo box to choose the activation function, sigmoid or tanh.

- text box to take the desired learning rate.

- text box to take the desired number of epochs.

- check box for the bias.

In [509]:
from tkinter import *

In [510]:
input_window = Tk()
input_window.title("Neural Networks Task 3")
input_window.geometry("500x500")
activation_fns = ["Sigmoid", "Tanh"]

<h3>Number of Hidden Layers</h3>

In [511]:
#Number of hidden Layers label
numOf_hidden_layers_value = StringVar()
numOf_hidden_layers_label = Label(input_window, textvariable = numOf_hidden_layers_value) 
numOf_hidden_layers_value.set("Number of hidden Layers")
numOf_hidden_layers_label.place(x=45, y=100)
#Number of hidden Layers text
numOf_hidden_layers_text = Entry(input_window)
numOf_hidden_layers_text.place(x=190, y=100)
numOf_hidden_layers_text.focus_set()

<h3>Number of neurons in each hidden layer</h3>

In [512]:
#Number of neurons hidden Layers label
numOf_neurons_value = StringVar()
numOf_neurons_label = Label(input_window, textvariable = numOf_neurons_value) 
numOf_neurons_value.set("Number of neurons in them")
numOf_neurons_label.place(x=30, y=140)
#Number of neurons hidden Layers text
numOf_neurons_text = Entry(input_window)
numOf_neurons_text.place(x=190, y=140)
numOf_neurons_text.focus_set()

<h3>Activation function Dropdown List</h3>

In [513]:
#Activation fn label
activation_fn_value = StringVar()
activation_fn_label = Label(input_window, textvariable = activation_fn_value) 
activation_fn_value.set("Activaion function")
activation_fn_label.place(x=80, y=170)
#Activation fn list
activation_fn_var = StringVar(input_window)
activation_fn = OptionMenu(input_window, activation_fn_var, *activation_fns)
activation_fn.config(width=12, font=('Helvetica', 10))
#activation_fn_var.set('Sigmoid') # set the default option
activation_fn.place(x=190, y=170)

<h3>Learning Rate</h3>

In [514]:
#learning rate label
learning_rate_value = StringVar()
learning_rate_label = Label(input_window, textvariable = learning_rate_value) 
learning_rate_value.set("Learning Rate")
learning_rate_label.place(x=105, y=210)
#learning rate text
learning_rate_text = Entry(input_window)
learning_rate_text.place(x=193, y=210)
learning_rate_text.focus_set()

<h3>Epochs</h3>

In [515]:
#Epochs label
epochs_label_value = StringVar()
epochs_label = Label(input_window, textvariable = epochs_label_value) 
epochs_label_value.set("Epochs")
epochs_label.place(x=140, y=240)
#Epochs text
epochs_text = Entry(input_window)
epochs_text.place(x=193, y=240)
epochs_text.focus_set()

<h3>Bias</h3>

In [516]:
#Bias check box
bias_checkbox = IntVar()
Checkbutton(input_window, text="Bias", variable=bias_checkbox).place(x=190,y=290)

<h3>Training The Model Button</h3>

In [517]:
def submit_button_backpropagation():
    layers_neurons = numOf_neurons_text.get().split()
    layers_neurons = [int(val) for val in layers_neurons]
    main(int(numOf_hidden_layers_text.get()), layers_neurons, activation_fn_var.get(), float(learning_rate_text.get()),
         int(epochs_text.get()), int(bias_checkbox.get()))


In [518]:
#Button
train_model_button = Button(input_window, text='Train backpropagation', width=17, command=submit_button_backpropagation)
train_model_button.place(x=190, y=320)

In [ ]:
input_window.mainloop() #open window

[{'weights': array([[0.11876262],
       [0.02701475],
       [0.41985974],
       [0.53088379],
       [0.94699367]]), 'inputs': array([[0.14072178],
       [0.30272609],
       [0.32956244],
       [0.10244711],
       [0.80673971]]), 'net': 0.2915485209042714, 'gradient': 0.09250016210488632}, {'weights': array([[0.83197294, 0.85250386, 0.75874892, 0.27086847, 0.89948105],
       [0.53153975, 0.67890424, 0.83330885, 0.99128863, 0.43000889],
       [0.20303106, 0.39891612, 0.32731053, 0.29146927, 0.24133346],
       [0.51087705, 0.17338538, 0.14506082, 0.86618465, 0.97314113],
       [0.03925007, 0.16949813, 0.54006018, 0.95635197, 0.06731636]]), 'inputs': array([[0.1048068 , 0.8521688 , 0.91715497, 0.42437696, 0.0785714 ],
       [0.35911262, 0.79289557, 0.63152992, 0.63947817, 0.98331047],
       [0.92082153, 0.53332395, 0.92132082, 0.17520147, 0.99507534],
       [0.10735221, 0.31411439, 0.30251394, 0.53594764, 0.12447308],
       [0.68624494, 0.75280091, 0.41812653, 0.86962729, 0

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Menna\Anaconda3\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-517-75233577d4ed>", line 5, in submit_button_backpropagation
    int(epochs_text.get()), int(bias_checkbox.get()))
  File "<ipython-input-508-18e3e383e7a4>", line 20, in main
    print('Confusion Matrix:\n', confusion_matrix(y_test, y_prediction_test))
  File "C:\Users\Menna\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 270, in confusion_matrix
    raise ValueError("%s is not supported" % y_type)
ValueError: multilabel-indicator is not supported
